In [ ]:
!pip install -q gspread

In [ ]:
import json 
from datetime import datetime

import gspread
import matplotlib.dates as mdates
import numpy as np
import pandas as pd
import plotly.graph_objects as go

In [ ]:
with open('config.json', 'r') as f:
    config = json.load(f)

In [ ]:
import boto3
from botocore.exceptions import ClientError

def get_secret():
    secret_name = config['serviceName']
    region_name = config['regionName']

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    # Decrypts secret using the associated KMS key.
    secret = get_secret_value_response['SecretString']
    secret_dict = json.loads(secret)

    return secret_dict

In [ ]:
current_month = int(datetime.now().strftime('%m'))
current_year = int(datetime.now().strftime('%Y'))

#connect to the service account
gc = gspread.service_account_from_dict(get_secret())

In [ ]:
file = gc.open(
    title=config['gSheetName']
)

df_list = []
for sheet in config['sheetNames']:
    curr_sheet = file.worksheet(sheet)
    curr_df = pd.DataFrame(curr_sheet.get_all_records(head=3))
    df_list.append(curr_df)

df = pd.concat(df_list)

In [ ]:
df = df[df['TYPE'] != 'Transfer']
df['YEAR'] = df['DATE'].str[:4].astype(int)
df['MONTH'] = df['DATE'].str[5:7].astype(int)
df.sort_values(by=['DATE'], inplace=True)
df.reset_index(drop=True, inplace=True)

## Spending per Month

In [ ]:
df_spendings = df[df['AMOUNT'] < 0].groupby(['YEAR', 'MONTH'])['AMOUNT'].sum().reset_index().tail(12)
df_spendings

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(
    x=[f"{year}/{month}" for year, month in zip(df_spendings['YEAR'], df_spendings['MONTH'])],
    y=df_spendings['AMOUNT'],
    width=0.1,
    text=df_spendings['AMOUNT'],
    textposition='auto'
))
fig.show()

## Cumsum per Month => Wallet value

In [ ]:
df['TREND'] = df['AMOUNT'].cumsum()

x = mdates.date2num(df['DATE'])
z = np.polyfit(
    x, 
    df['TREND'], 
    1
)
p = np.poly1d(z)

df['TREND_VAL'] = p(x)

In [ ]:
df_trend = df.groupby(['DATE']).agg({'TREND': 'max', 'TREND_VAL': 'max'}).reset_index()
slope = np.round(
    z[0] / df_trend['TREND'].iloc[0] * 100,
    2
)

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df_trend['DATE'],
    y=df_trend['TREND'],
    name='cumsum'
))

fig.add_trace(go.Scatter(
    x=df_trend['DATE'],
    y=df_trend['TREND_VAL'],
    name=f'trendline {slope}'
))
fig.show()

### Cumsum per month

In [ ]:
df_cumsum_month = df.groupby(['YEAR', 'MONTH'])['AMOUNT'].sum().groupby(level=0).cumsum().reset_index().tail(12)
df_cumsum_month

## Spendig per Category in the last month

In [ ]:
df_category = df[(df['YEAR'] == current_year) & (df['MONTH'] == current_month) & (df['AMOUNT'] < 0)].groupby('TYPE')['AMOUNT'].sum().reset_index()
df_category['AMOUNT'] = df_category['AMOUNT'] * -1

df_category.sort_values(by='AMOUNT', ascending=False).reset_index(drop=True)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Pie(
    values=df_category['AMOUNT'],
    labels=df_category['TYPE']
))
fig.show()